<a href="https://colab.research.google.com/github/bbberylll/ESAA_OB/blob/main/%EC%88%98%EC%83%81%EC%9E%91%EB%A6%AC%EB%B7%B0_0919%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[American Express - Default Prediction](https://www.kaggle.com/competitions/amex-default-prediction?utm_source=chatgpt.com)

## 대회 개요
1. 목표: 신용카드 고객의 12개월 내 디폴트(default) 여부 예측 (이진 분류)

2. 데이터셋
    * 고객 정보 (static features): demographic, card/account 관련 속성
    * 거래 이력 (dynamic features): 월별 청구/결제/신용한도 등 시계열 데이터
    * 데이터 크기: 약 5.5GB, 1.2억 행, 50만 명 이상 고객
3. 평가지표: Amex Metric (특수 설계된 지표, AUC + Top-4% precision을 조합)



## 공통 전략
1. 시계열 --> 탭형 요약
    * 고객별 여러 달(month)의 데이터를 집계(aggregation) → lag, rolling, trend 변수 생성
    * 예: max_balance, min_payment_ratio, std_utilization, last_month_trend

2. 불균형 데이터 처리
    * default 비율이 약 4~5% --> class_weight, focal loss, downsampling 활용

3. 모델링
    * 대부분 LightGBM, CatBoost, XGBoost 같은 GBDT 계열
    * 일부 팀은 Neural Net (Transformer, TabNet) 시도, 하지만 부스팅이 더 강력

4. 앙상블
    * 단일 모델보다 스태킹(Stacking), 블렌딩(Blending), Rank Averaging 사용
    * CV 안정성과 Public/Private LB 차이 줄이기 위해 seed 다양화 + 교차검증 반복

## 1등 팀
1. **“고객별 temporal aggregation”**에 집중: 최근 n개월, 중간 기간, 전체 기간 요약
2. 각 시계열 피처에 대해 mean, std, min, max, trend(기울기) 생성
3. Cross-validation 설계: 고객 단위 CV (데이터 누수 방지)
4. 모델링: LightGBM + XGBoost 앙상블
5. 차별점: Amex Metric과 잘 align되는 custom loss 설계

## 2등 팀
1. Neural Net 기반 시도(Transformer-like) + LightGBM
2. NN 단독보다 GBDT 성능이 높아 앙상블에서 보조 역할로 NN 포함
3. 데이터 정규화, embedding layer 적용
4. tabular + sequence hybrid 모델


## 코드 분석
### EDA
* target 분포 확인 (불균형)
* 고객별 row 개수(기간별 데이터 길이 분포)
* 주요 시계열 피처의 시각화 (예: balance, utilization)

### 전처리
* 고객 ID 기준 groupby → 시계열 집계 (agg: mean, std, min, max, last)
* 결측치 처리: CatBoost/LGBM은 NaN 허용, 필요시 fillna
* 범주형 변수는 CatBoost 자동 처리 or one-hot encoding

### 특성 공학
* lag/rolling 평균, 마지막 달 값, 최근 증가율(trend)
* ratio 변수: payment/total_balance, utilization = balance/limit
* customer-level statistics

### 모델링
* LightGBM, XGBoost, CatBoost (GPU 가속)
* Class imbalance 대응 → scale_pos_weight (XGB/LGBM), class_weights (CatBoost)
* Custom loss: logloss + amex metric surrogate

### 앙상블
* 여러 seed, 여러 boosting 모델 결과 rank averaging
* 일부는 NN 출력까지 blending

### 평가
* Amex metric 계산 함수 구현 (LB와 동일)
* CV 결과 vs Public/Private LB 차이 확인

## 배울 점
1. 시계열 요약 → 탭형 변환: 이 대회의 가장 큰 포인트. 원본 시계열을 그대로 쓰기보다 의미 있는 집계 변수로 바꾸는 것이 핵심

2. 데이터 누수 방지: 고객 단위 split 필수 (같은 고객 train/test 겹치면 leakage)

3. 앙상블의 힘: boosting 단독도 강력하지만, 여러 모델을 blending/stacking하면 더 안정적

4. 지표 적합화(Custom loss): 단순 AUC가 아니라, 대회 전용 metric(Amex metric)을 반영하는 학습 전략이 필요